In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 23.4 MB/s eta 0:00:00


In [3]:
!pip install duckdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 56.0 MB/s eta 0:00:00


In [4]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 kB 42.6 MB/s eta 0:00:00


In [7]:
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 3.0 MB/s eta 0:00:00


In [8]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
import numpy as np
import optuna

In [9]:
import duckdb
import pandas as pd

train_path = '/content/drive/MyDrive/BELKA-DrugPredict/data/train.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 100000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 100000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [10]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,212776275,O=C(Nc1cccc(F)c1C(=O)O)OCC1c2ccccc2-c2ccccc21,CSc1ccc(CN)o1.Cl,Cc1cc(N)nnc1Cl,CSc1ccc(CNc2nc(Nc3cc(C)c(Cl)nn3)nc(Nc3cccc(F)c...,BRD4,0
1,255235707,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,COc1ncc(N)cc1C(F)(F)F,NCc1nc2ccccc2s1,COc1ncc(Nc2nc(NCc3nc4ccccc4s3)nc(N[C@H](CC(=O)...,BRD4,0
2,119620510,O=C(N[C@H](Cc1ccc(-c2ccccc2)cc1)C(=O)O)OCC1c2c...,Nc1cc(N2CCNCC2)ccc1[N+](=O)[O-],C=CCOCCCN,C=CCOCCCNc1nc(Nc2cc(N3CCNCC3)ccc2[N+](=O)[O-])...,HSA,0
3,184162537,O=C(Nc1ccc(C(=O)O)c(Cl)c1)OCC1c2ccccc2-c2ccccc21,CC(C)(CN)CCC#N,CN1CCN(c2ccc(N)cc2F)CC1,CN1CCN(c2ccc(Nc3nc(NCC(C)(C)CCC#N)nc(Nc4ccc(C(...,HSA,0
4,55801922,Cc1ccc(C(CC(=O)O)NC(=O)OCC2c3ccccc3-c3ccccc32)cc1,COc1cc(Br)ccc1N,NCc1nc2ccccc2s1,COc1cc(Br)ccc1Nc1nc(NCc2nc3ccccc3s2)nc(NC(CC(=...,sEH,0


In [11]:
# ECFP 생성 함수
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

# 분자의 물리화학적 특성을 추출하는 함수
def generate_physchem_features(molecule):
    if molecule is None:
        return [np.nan] * 5
    return [
        Descriptors.MolWt(molecule),
        Descriptors.MolLogP(molecule),
        Descriptors.NumHDonors(molecule),
        Descriptors.NumHAcceptors(molecule),
        Descriptors.TPSA(molecule)
    ]

In [12]:
# 데이터 전처리
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)
df['ecfp'] = df['molecule'].apply(generate_ecfp)
df['physchem'] = df['molecule'].apply(generate_physchem_features)

In [13]:
# ECFP와 물리화학적 특성을 결합
df['features'] = df.apply(lambda row: row['ecfp'] + row['physchem'], axis=1)

In [14]:
# 단백질 이름을 원-핫 인코딩
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

## Train

In [15]:
# 최종 입력 데이터 생성
X = [features + list(protein) for features, protein in zip(df['features'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

In [16]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Trial 49 finished with value: 0.9582606969153915 and parameters: {'n_estimators': 300, 'max_depth': 8, 'learning_rate': 0.01461751822935858, 'subsample': 0.6594401898016142, 'colsample_bytree': 0.7589525793147899}. Best is trial 47 with value: 0.9790948500458416.

In [17]:
# 최적의 하이퍼파라미터로 모델 생성 및 학습
best_params = {
    'n_estimators': 300,
    'max_depth': 8,
    'learning_rate': 0.01461751822935858,
    'subsample': 0.6594401898016142,
    'colsample_bytree': 0.7589525793147899
}
best_model = XGBClassifier(eval_metric='logloss', use_label_encoder=False, **best_params)
best_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7589525793147899, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01461751822935858,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

## Submission

In [18]:
# 테스트 데이터 예측 및 저장
test_file = '/content/drive/MyDrive/BELKA-DrugPredict/data/test.csv'
output_file = '/content/drive/MyDrive/BELKA-DrugPredict/output/submission1.csv'

In [21]:
import os

In [22]:
for df_test in pd.read_csv(test_file, chunksize=200000):
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)
    df_test['physchem'] = df_test['molecule'].apply(generate_physchem_features)
    df_test['features'] = df_test.apply(lambda row: row['ecfp'] + row['physchem'], axis=1)
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))
    X_test = [features + list(protein) for features, protein in zip(df_test['features'].tolist(), protein_onehot.tolist())]
    probabilities = best_model.predict_proba(X_test)[:, 1]
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))